In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

train_transform = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean, std),
])

test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean, std),
])


In [6]:
trainset = torchvision.datasets.CIFAR10(
    root="./../datasets", train=True, transform=train_transform
)
testset = torchvision.datasets.CIFAR10(
    root="./../datasets", train=False, transform=test_transform
)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)
classes = trainset.classes


In [7]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_chans=3, embed_dim=128):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid = img_size // patch_size
        self.num_patches = self.grid * self.grid
        self.proj = nn.Conv2d(in_chans, embed_dim,
                              kernel_size=patch_size,
                              stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        return x


In [20]:
class ViT(nn.Module):
    def __init__(self,
                 img_size=32,
                 patch_size=4,
                 in_chans=3,
                 num_classes=10,
                 embed_dim=128,
                 depth=8,
                 num_heads=8,
                 mlp_ratio=4.0,
                 drop_rate=0.1):
        super().__init__()

        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(drop_rate)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=int(embed_dim * mlp_ratio),
            dropout=drop_rate,
            batch_first=True,
            activation='gelu'
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        x = self.encoder(x)
        x = self.norm(x)
        cls_out = x[:, 0]
        logits = self.head(cls_out)
        return logits


In [9]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / total, correct / total


In [10]:
def evaluate(model, loader, criterion, device):
    model.eval()
    loss_sum, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_sum += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return loss_sum / total, correct / total


In [21]:
model = ViT().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

num_epochs = 15
best_val = 0.0
patience, wait = 3, 0

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, trainloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, testloader, criterion, device)
    scheduler.step()

    if val_acc > best_val:
        best_val = val_acc
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping")
            break

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
          f"Test loss: {val_loss:.4f}, acc: {val_acc:.4f}")


Epoch 1/15 Train loss: 1.8019, acc: 0.3217 | Test loss: 1.6184, acc: 0.4148
Epoch 2/15 Train loss: 1.4625, acc: 0.4645 | Test loss: 1.3462, acc: 0.5182
Epoch 3/15 Train loss: 1.2794, acc: 0.5346 | Test loss: 1.2319, acc: 0.5551
Epoch 4/15 Train loss: 1.1611, acc: 0.5807 | Test loss: 1.1446, acc: 0.5837
Epoch 5/15 Train loss: 1.0705, acc: 0.6149 | Test loss: 0.9870, acc: 0.6449
Epoch 6/15 Train loss: 0.9942, acc: 0.6445 | Test loss: 0.9751, acc: 0.6470
Epoch 7/15 Train loss: 0.9430, acc: 0.6626 | Test loss: 0.9089, acc: 0.6801
Epoch 8/15 Train loss: 0.8858, acc: 0.6845 | Test loss: 0.8573, acc: 0.6938
Epoch 9/15 Train loss: 0.8373, acc: 0.7016 | Test loss: 0.8046, acc: 0.7133
Epoch 10/15 Train loss: 0.7952, acc: 0.7168 | Test loss: 0.7816, acc: 0.7230
Epoch 11/15 Train loss: 0.7636, acc: 0.7310 | Test loss: 0.7586, acc: 0.7263
Epoch 12/15 Train loss: 0.7365, acc: 0.7398 | Test loss: 0.7363, acc: 0.7391
Epoch 13/15 Train loss: 0.7146, acc: 0.7473 | Test loss: 0.7277, acc: 0.7435
Epoch 14

In [24]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f'\nModel Accuracy: {100 * correct / total:.2f} %')
  print('Correct:', correct, 'Total:', total)


Model Accuracy: 74.53 %
Correct: 7453 Total: 10000
